### pytorch notebook for reweighting using density ratio estimation with calibrated classifiers

The idea behind this notebook is to reweight one distribution $p_0(x)$ to look like another distribution $p_1(x)$.  

The reweighting technique is based on [Approximating Likelihood Ratios with Calibrated Discriminative Classifiers](http://inspirehep.net/record/1377273). 

In this notebook V+jets samples generated with Madgraph5 ($p_0(x)$) and Sherpa ($p_1(x)$) are compared, and the  weights are derived to reweight Madgraph5 to look like Sherpa.  

The performance of the weights, i.e. how well the reweighted original distribution matches the target distribution, is assessed by training a discriminator to differentiate the original distribution with weights applied from a target distribution.  

Work in progress by Leonora Vesterbacka. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import theano
from itertools import product
import root_numpy
import pandas as pd
import uproot
import torch
np.random.seed(314)

Welcome to JupyROOT 6.18/00


In [2]:
#the example has much more real data than monte carlo
#this leads to unbalanced dataset
#some techniques deal with that better than others
data_to_use = ["all","max balanced"][0]

# the histogram and kde calibration don't work very well
#with very peaked output score distributions,
#but the isotonic approach does
calibration_type = ["isotonic", "kde", "histogram"][0]

#do either training using all phase space by defining do = "varAll", or just two variables by defining do = "var2"
do = ["two", "all"][0]
normalize = False

In [3]:
if do == "two":
    binning = [range(0, 2400, 200), range(0, 15, 1)]
    variables = ['VpT','Njets']
    vlabels = ['V $\mathrm{p_{T}}$ [GeV]','Number of jets']

    weights = ['normweight']
if do == "all":
    etaV = [-10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    etaJ = [-2.8,-2.4,-2,-1.6,-1.2,-0.8,-0.4,0,0.4,0.8,1.2,1.6,2,2.4,2.8]
    variables = ['VpT','Njets','j1pT', 'j2pT', 'HT','ptmiss', 'l1pT','Veta','j1eta','j2eta']
    vlabels = ['V $\mathrm{p_{T}}$ [GeV]','Number of jets','Leading jet $\mathrm{p_{T}}$ [GeV]','Subleading jet $\mathrm{p_{T}}$ [GeV]', '$\mathrm{H_{T}}$ [GeV]','$\mathrm{p_{T}^{miss}}$ [GeV]', 'Leading lepton $\mathrm{p_{T}}$ [GeV]','V $\eta$','Leading jet $\eta$','Subleading jet $\eta$']
    binning = [range(0, 2400, 100), range(0, 15, 1), range(0, 2700, 100),range(0, 2700, 100),range(0, 4000, 200),range(0, 600, 50),range(0, 1500, 50), etaV, etaJ, etaJ]
    weights = ['normweight']
    #truthWeight is the generator weight
#get original and target samples, madgraph:original, sherpa:target
original  = root_numpy.root2array('/eos/user/m/mvesterb/data/madgraph/one/Nominal.root', branches=variables)
target    = root_numpy.root2array('/eos/user/m/mvesterb/data/sherpa/one/Nominal.root', branches=variables)
#originalW = root_numpy.root2array('/eos/user/m/mvesterb/data/madgraph/one/Nominal.root', branches=weights)
#targetW   = root_numpy.root2array('/eos/user/m/mvesterb/data/sherpa/one/Nominal.root', branches=weights)
#create dataframes to do the training on, and also get the sample weights in separate dataframes for resampling
oDF   = pd.DataFrame(original,columns=variables)
tDF   = pd.DataFrame(target,columns=variables)
#oWDF  = pd.DataFrame(originalW,columns=weights)
#tWDF  = pd.DataFrame(targetW,columns=weights)

oT = torch.tensor(oDF[variables].values,dtype=torch.float)
tT = torch.tensor(tDF[variables].values,dtype=torch.float)
print("oT",oT)

('oT', tensor([[51.6451,  1.0000],
        [ 2.4802,  0.0000],
        [14.1178,  0.0000],
        ...,
        [71.5885,  0.0000],
        [ 9.1785,  0.0000],
        [10.2159,  0.0000]]))


A discriminator is trained to differentiate the original and the target distributions from each other, as well as differentiating the target distribution from the original distributions with the learned carl applied. Well learned weights would make the target and reweighted distributions very similar and indistinguishable for the discriminator. 

In [19]:
#to randomize training and test data
n_target = tT.shape[0]
rand = np.random.choice(range(oT.shape[0]),2*n_target,replace=True)
randomized_original = oT[rand]

X0_all = randomized_original[:n_target,:]
X0_test = randomized_original[-n_target:,:]

X1_all = tT
print(oT.shape)
print(X0_all.shape)
print(X0_test.shape)
print(X1_all.shape)


torch.Size([20981, 2])
torch.Size([83316, 2])
torch.Size([83316, 2])
torch.Size([83316, 2])


In [5]:
#make training data from all samples
num1 = X0_all.shape[0]
num2 = X1_all.shape[0]

#X_all = np.vstack((X0_all,X1_all))
#y_all = np.ones(num1 + num2, dtype=np.int)
X_all = torch.cat([X0_all,X1_all])
y_all = torch.ones(num1+num2,dtype=torch.float) 
y_all[num1:] = 0

#randomly sample X0 to have the same number of entries as X1
# assuming X0 is bigger here
X0_s = X0_all[np.random.choice(range(X0_all.shape[0]),num1,replace=True)]
X_s = torch.cat((X0_s, X1_all))
y_s = torch.ones(num1 + num2, dtype=torch.int)
y_s[num1:] = 0

X1_x = X1_all[np.random.choice(range(X1_all.shape[0]),num1,replace=True)]
X_x = torch.cat((X0_all, X1_x))
y_x = torch.ones(num1 + num1, dtype=torch.int)
y_x[num1:] = 0
#now use the flags to decide which of the datasets to use
X, X0, X1, y = None, None, None, None
if data_to_use == "all":
    X, X0, X1, y = X_all, X0_all, X1_all, y_all
elif data_to_use == "max balanced":
    X, X0, X1, y = X_s, X0_s, X1_all, y_s
else:    
    print("error")
X0.requires_grad_(True)
X1.requires_grad_(True)
X.requires_grad_(True)
print("X",X)
X = torch.FloatTensor(X)
X0 = torch.FloatTensor(X0)
X1 = torch.FloatTensor(X1)
y = torch.FloatTensor(y)

print(X0.shape)
print(X1.shape)
print(y.shape)
print(X.shape)


('X', tensor([[ 7.3920,  0.0000],
        [ 6.8687,  1.0000],
        [34.5983,  0.0000],
        ...,
        [37.7138,  1.0000],
        [10.6486,  0.0000],
        [32.3756,  1.0000]], requires_grad=True))
torch.Size([83316, 2])
torch.Size([83316, 2])
torch.Size([166632])
torch.Size([166632, 2])


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
## Define the NN architecture
class Feedforward(torch.nn.Module):
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.hidden_size, 1)
            self.sigmoid = torch.nn.Sigmoid()
        def forward(self, x):
            hidden = self.fc1(x)
            relu = self.relu(hidden)
            output = self.fc2(relu)
            output = self.sigmoid(output)
            return output


In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils_data
from torch.autograd import Variable

inputs = Variable(X)
targets = Variable(y)

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out = self.fc2(F.relu(self.fc1(x)))
        return out

input_size = inputs.size()[1]
hidden_size = 100
output_size = 1
num_epoch = 200
learning_rate = 1e-3
model = MLP(input_size = input_size, hidden_size = hidden_size,
            output_size = output_size)

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=1e-4)
loss_fct = nn.MSELoss()
training_samples = utils_data.TensorDataset(inputs, targets)
data_loader_trn = utils_data.DataLoader(training_samples, batch_size=200, drop_last=False, shuffle=True)

#train
for epoch in range(num_epoch):
        cum_loss = 0
        for batch_idx, (data, target) in enumerate(data_loader_trn):

            X, y = data.float(), target.float()

            pred = model(X)
            loss = loss_fct(pred, y.unsqueeze(1)) 

        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            cum_loss += loss.item()
            
        if epoch % 10 == 0:
            print ('Epoch [%d/%d], Loss: %.4f' 
                      %(epoch+1, num_epoch, cum_loss))


final_prediction = model(inputs)
final_pred_np = final_prediction.clone().detach().numpy()

np.corrcoef(final_pred_np.squeeze(), targets)[0,1]

Epoch [1/200], Loss: 215.5257
Epoch [11/200], Loss: 204.2476
Epoch [21/200], Loss: 203.8199
Epoch [31/200], Loss: 204.2938
Epoch [41/200], Loss: 203.3584
Epoch [51/200], Loss: 202.9949
Epoch [61/200], Loss: 202.6688
Epoch [71/200], Loss: 202.4050
Epoch [81/200], Loss: 202.0863
Epoch [91/200], Loss: 201.8631
Epoch [101/200], Loss: 201.9190
Epoch [111/200], Loss: 201.6627
Epoch [121/200], Loss: 201.6057
Epoch [131/200], Loss: 201.6184
Epoch [141/200], Loss: 201.4360
Epoch [151/200], Loss: 201.3977
Epoch [161/200], Loss: 201.4014
Epoch [171/200], Loss: 201.3519
Epoch [181/200], Loss: 201.2463
Epoch [191/200], Loss: 201.2540


0.18802762862010208

In [7]:
model = Feedforward(2, 10)
# specify loss function
criterion = torch.nn.BCELoss()
# specify optimizer
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [8]:
model.eval()
print("model",model)
y_pred = model(X)
before_train = criterion(y_pred.squeeze(), y)
print('Test loss before training' , before_train.item())

('model', Feedforward(
  (fc1): Linear(in_features=2, out_features=10, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=10, out_features=1, bias=True)
  (sigmoid): Sigmoid()
))
('Test loss before training', 2.37048077583313)


In [10]:
model.train()
epoch = 20
for epoch in range(epoch):
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(X)
    # Compute Loss
    loss = criterion(y_pred.squeeze(), y)
   
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()

Epoch 0: train loss: 0.693524181843
Epoch 1: train loss: 0.693477332592
Epoch 2: train loss: 0.693429529667
Epoch 3: train loss: 0.693384587765
Epoch 4: train loss: 0.693343222141
Epoch 5: train loss: 0.693293750286
Epoch 6: train loss: 0.693261802197
Epoch 7: train loss: 0.693234980106
Epoch 8: train loss: 0.693212747574
Epoch 9: train loss: 0.693188965321
Epoch 10: train loss: 0.693153619766
Epoch 11: train loss: 0.693126380444
Epoch 12: train loss: 0.693103075027
Epoch 13: train loss: 0.693069756031
Epoch 14: train loss: 0.69303894043
Epoch 15: train loss: 0.693009018898
Epoch 16: train loss: 0.692978978157
Epoch 17: train loss: 0.692942619324
Epoch 18: train loss: 0.692906439304
Epoch 19: train loss: 0.692872643471


In [12]:
model.eval()
y_pred = model(X)
after_train = criterion(y_pred.squeeze(), y) 
print('Test loss after Training' , after_train.item())
print("y_pred"), y_pred

('Test loss after Training', 0.6928380131721497)
y_pred tensor([[0.4552],
        [0.4532],
        [0.4404],
        ...,
        [0.4315],
        [0.4534],
        [0.4344]], grad_fn=<SigmoidBackward>)
